# Simple problem -- single city - single order




In [1]:
import pandas as pd
import pulp as plp

In [2]:
input_file = "../data/test_input.xlsx"
output_file = "../data/test_output.xlsx"

## Data

In [3]:
sheet_name = "Parameters"
df_param = pd.read_excel(input_file, sheet_name=sheet_name)
df_param.set_index("ParamName", inplace=True)
minPeriod = df_param.loc["minPeriod"].ParamValue
maxPeriod = df_param.loc["maxPeriod"].ParamValue
minPeriod, maxPeriod

(0, 25)

In [4]:
sheet_name = "SimpleOrders"
#sheet_name = "Orders"
df_orders = pd.read_excel(input_file, sheet_name=sheet_name)
df_orders.head()

,ID,City,Demand,Price_per_unit,Deliver_from,Deliver_to
0,1,Tambov,200,80,3,13
1,2,Voronezh,100,110,17,24


In [5]:
sheet_name = "Routes"
df_routes = pd.read_excel(input_file, sheet_name=sheet_name)
df_routes.head()

,ID,From,To,Time,Distance
0,1,Moscow,Tambov,4,500
1,2,Moscow,Voronezh,3,400
2,3,Tambov,Moscow,4,500
3,4,Tambov,Voronezh,2,200
4,5,Voronezh,Moscow,3,400


In [6]:
sheet_name = "Cities"
df_cities = pd.read_excel(input_file, sheet_name=sheet_name)
df_cities.head()

,ID,City
0,1,Moscow
1,2,Tambov
2,3,Voronezh


In [7]:
sheet_name = "Cars"
df_cars= pd.read_excel(input_file, sheet_name=sheet_name, nrows=None)
df_cars.head()

,ID,CarName,Capacity,Price_per_km
0,1,big,60,10
1,2,small,50,8


In [8]:
TOT_CAPACITY = df_cars.Capacity.sum()
TOT_CAPACITY

110

## Structures

Moscow - origin depot and destination depot

In [9]:
def convert_dataframe_to_dict(dataframe, key_columns, value_column):
    return (
        dataframe.loc[:, key_columns + [value_column]]
        .set_index(key_columns)
        .to_dict()[value_column]
    )

def convert_dataframe_to_set(dataframe, key_columns, value_column):
    return set(
        list(convert_dataframe_to_dict(dataframe, key_columns, value_column).keys())
    )


In [10]:
depot = df_cities.City[0]

cities = set(df_cities.City.values).difference(set([depot]))

#origin_depot = default_depot + '_start'
#destination_depot = default_depot + '_end'

all_cities = cities.union([depot])
all_cities

{'Moscow', 'Tambov', 'Voronezh'}

In [11]:
depot

'Moscow'

In [12]:
car_capacity = convert_dataframe_to_dict(df_cars, ["CarName"], "Capacity")
car_capacity

{'big': 60, 'small': 50}

In [13]:
car_price = convert_dataframe_to_dict(df_cars, ["CarName"], "Price_per_km")
car_price

{'big': 10, 'small': 8}

In [14]:
routes = convert_dataframe_to_set(df_routes, ["From", "To"], "Distance")
routes

{('Moscow', 'Tambov'),
 ('Moscow', 'Voronezh'),
 ('Tambov', 'Moscow'),
 ('Tambov', 'Voronezh'),
 ('Voronezh', 'Moscow'),
 ('Voronezh', 'Tambov')}

In [15]:
route_time = convert_dataframe_to_dict(df_routes, ["From", "To"], "Time")
route_distance = convert_dataframe_to_dict(df_routes, ["From", "To"], "Distance")

In [16]:
route_time, route_distance, routes

({('Moscow', 'Tambov'): 4,
  ('Moscow', 'Voronezh'): 3,
  ('Tambov', 'Moscow'): 4,
  ('Tambov', 'Voronezh'): 2,
  ('Voronezh', 'Moscow'): 3,
  ('Voronezh', 'Tambov'): 2},
 {('Moscow', 'Tambov'): 500,
  ('Moscow', 'Voronezh'): 400,
  ('Tambov', 'Moscow'): 500,
  ('Tambov', 'Voronezh'): 200,
  ('Voronezh', 'Moscow'): 400,
  ('Voronezh', 'Tambov'): 200},
 {('Moscow', 'Tambov'),
  ('Moscow', 'Voronezh'),
  ('Tambov', 'Moscow'),
  ('Tambov', 'Voronezh'),
  ('Voronezh', 'Moscow'),
  ('Voronezh', 'Tambov')})

In [17]:
orders = df_orders.drop("ID", axis=1).set_index("City")
orders

,Demand,Price_per_unit,Deliver_from,Deliver_to
City,,,,
Tambov,200,80,3,13
Voronezh,100,110,17,24


In [18]:
TOT_DEMAND = df_orders.Demand.sum()
TOT_DEMAND

300

In [19]:
cars = convert_dataframe_to_set(df_cars, ["CarName"], "Price_per_km")
cars

{'big', 'small'}

In [20]:
# trips - number of trips performed by one vehicle
trips = list(range(1, len(cities)))
trips = list(range(1, TOT_DEMAND//TOT_CAPACITY + 2))
fin_trip = trips[-1]
trips, fin_trip

([1, 2, 3], 3)

## Task

k - cars

n - trips

r - routes

$$
x^{kn}_{ij} = 1 if trip n \in TRIPS of vehicle v \in V travels through arc (i,j) \in A, 0 otherwise,
$$

In [21]:
pbm = plp.LpProblem("VRPTW", plp.LpMaximize)

In [22]:
x = plp.LpVariable.dicts("x", [(k, n, r) for k in cars for n in trips for r in routes], cat=plp.LpBinary)

In [23]:
y = plp.LpVariable.dicts("y", [(k, n, i) for k in cars for n in trips for i in all_cities], cat=plp.LpBinary)

In [24]:
d = plp.LpVariable.dicts("d", [(k, n, i) for k in cars for n in trips for i in cities], cat=plp.LpContinuous, lowBound=0)

In [25]:
#lack
lack = plp.LpVariable.dicts("lack", [i for i in cities], cat=plp.LpContinuous, lowBound=0)

In [26]:
t = plp.LpVariable.dicts("t", [(k, n, i) for k in cars for n in trips for i in all_cities], cat=plp.LpContinuous, lowBound=0)

In [27]:
# end time of the trip n
te = plp.LpVariable.dicts("te", [(k, n) for k in cars for n in trips], cat=plp.LpContinuous, lowBound=0)

In [28]:
# maximize revenue
#revenue = sales_profit - devlivery_cost + expensive_undelivered
pbm += plp.lpSum(orders.loc[i].Price_per_unit * d[k, n, i] for k in cars for n in trips for i in cities) \
    - plp.lpSum(route_distance[r] * car_price[k] * x[k, n, r] for k in cars for n in trips for r in routes) \
    - plp.lpSum(te[k, n] for k in cars for n in trips) \
    - plp.lpSum(100 * orders.loc[i].Price_per_unit * lack[i] for i in cities)

minimize travel time
```python
pbm += plp.lpSum(route_time[r] * x[k, n, r] for k in cars for n in trips for r in routes)
```

minimize distance cost
```python
prob += plp.lpSum(route_distance[r] * car_price[k] * x[k, n, r] for k in cars for n in trips for r in routes)
```

In [30]:
# time constraints
for k in cars:
    pbm += t[k, 1, depot] == 0
    for n in trips:
        if n != fin_trip:
            pbm += te[k, n] <= t[k, n+1, depot]
            pbm += t[k, n, depot] <= t[k, n+1, depot]
        pbm += te[k, n] <= maxPeriod
        
        for i, j in routes:
            if j == depot:
                continue
            pbm += t[k, n, i] + route_time[i, j] <= t[k, n, j] + maxPeriod * (1 - x[k, n, (i, j)])
        for i in cities:
            pbm += t[k, n, i] + route_time[i, depot] <= te[k, n] + maxPeriod * (1 - x[k, n, (i, depot)])
            pbm += t[k, n, i] >= orders.loc[i].Deliver_from * y[k, n, i]
            pbm += t[k, n, i] <= orders.loc[i].Deliver_to * y[k, n, i]

In [31]:
for i in cities:
    # every city visited at least once by one vehicle
    pbm += plp.lpSum(y[k, n, i] for k in cars for n in trips) >= 1

    # demand
    pbm += plp.lpSum(d[k, n, i] for k in cars for n in trips) == orders.loc[i].Demand - lack[i]
    
    
for k in cars:
    for i in all_cities:
        for n in trips:
            pbm += plp.lpSum(x[k, n, (i, j)] for j in all_cities if i != j) == y[k, n, i]
            pbm += plp.lpSum(x[k, n, (j, i)] for j in all_cities if i != j) == y[k, n, i]
    
    for n in trips:
        pbm += plp.lpSum(d[k, n, i] for i in cities) <= car_capacity[k] 
        for i in cities:
            pbm += d[k, n, i] <= car_capacity[k] * y[k, n, i] 

In [32]:
pbm.solve()

1

In [33]:
plp.LpStatus[pbm.status], plp.value(pbm.objective)

('Optimal', -23492.0)

In [34]:
revenue = sum(orders.loc[i].Price_per_unit * d[k, n, i].value() for k in cars for n in trips for i in cities) \
    - sum(route_distance[r] * car_price[k] * x[k, n, r].value() for k in cars for n in trips for r in routes)
revenue

-23400.0

In [35]:
#pbm

## Save result

In [36]:
cols = ["CarName",
        "TripNumber",
        "City_From", 
        "City_To", 
        "Delivered",
        "Lack",
        "StartTime",
        "ArrivalTime", "Deliver_From", "Deliver_To"]
result = pd.DataFrame(columns=cols)
row_number = 0
for k in cars:
    for n in trips:
        for i, j in routes:
            #print(k, n, i, j, x[k, n, (i, j)].value())
            if x[k, n, (i, j)].value() == 0:
                continue
            result.loc[row_number, cols] = k, n, i, j, \
                                        d[k, n, j].value() if j != depot else None, \
                                        lack[j].value() if j != depot else None, \
                                         t[k, n, i].value(), \
                                         t[k, n, j].value() if j != depot else te[k, n].value(), \
                                         orders.loc[j].Deliver_from if j in cities else None, \
                                         orders.loc[j].Deliver_to if j in cities else None
            row_number += 1

            
result.sort_values(by=["CarName", "TripNumber", "ArrivalTime"])
#result

,CarName,TripNumber,City_From,City_To,Delivered,Lack,StartTime,ArrivalTime,Deliver_From,Deliver_To
1,big,1,Moscow,Tambov,60,0,0,4,3,13
0,big,1,Tambov,Moscow,None,None,4,8,None,None
3,big,2,Moscow,Tambov,60,0,8,12,3,13
2,big,2,Tambov,Moscow,None,None,12,16,None,None
5,big,3,Moscow,Voronezh,60,0,16,19,17,24
4,big,3,Voronezh,Moscow,None,None,19,22,None,None
7,small,1,Moscow,Tambov,50,0,0,4,3,13
6,small,1,Tambov,Moscow,None,None,4,8,None,None
9,small,2,Moscow,Tambov,30,0,8,12,3,13
8,small,2,Tambov,Moscow,None,None,12,16,None,None


In [37]:
for k in cars:
    for n in trips:
        print(k, n, te[k, n].value())
        for i in all_cities:
            print(k, n, i, t[k, n, i].value(), y[k, n, i].value())

big 1 8.0
big 1 Voronezh 0.0 0.0
big 1 Tambov 4.0 1.0
big 1 Moscow 0.0 1.0
big 2 16.0
big 2 Voronezh 0.0 0.0
big 2 Tambov 12.0 1.0
big 2 Moscow 8.0 1.0
big 3 22.0
big 3 Voronezh 19.0 1.0
big 3 Tambov 0.0 0.0
big 3 Moscow 16.0 1.0
small 1 8.0
small 1 Voronezh 0.0 0.0
small 1 Tambov 4.0 1.0
small 1 Moscow 0.0 1.0
small 2 16.0
small 2 Voronezh 0.0 0.0
small 2 Tambov 12.0 1.0
small 2 Moscow 8.0 1.0
small 3 22.0
small 3 Voronezh 19.0 1.0
small 3 Tambov 0.0 0.0
small 3 Moscow 16.0 1.0
